# Pobieranie obrazów satelitarnych

## Import bibliotek

In [ ]:
from planet import Planet, data_filter, order_request
from datetime import datetime, timedelta
import json

# Uwierzytelnienie (ustaw zmienną środowiskową)
PL_API_KEY='PLAK909f0895d2484280aefe9612a961fc34'
pl = Planet()  # automatycznie wykrywa PL_API_KEY

AttributeError: 'str' object has no attribute '_client'

## Ustawienie obszaru

In [2]:
remedhus_aoi = {
    "type": "Polygon",
    "coordinates": [[
        [-5.7, 41.1],  # SW corner
        [-5.1, 41.1],  # SE corner
        [-5.1, 41.5],  # NE corner
        [-5.7, 41.5],  # NW corner
        [-5.7, 41.1]   # zamknięcie poligonu
    ]]
}

# Lub wczytaj z pliku GeoJSON
# with open('remedhus_aoi.geojson', 'r') as f:
#     remedhus_aoi = json.load(f)['features'][0]['geometry']

## Wyszukiwanie obrazów

In [4]:
def search_daily_imagery(start_date, end_date, aoi, max_cloud_cover=0.2):
    """
    Wyszukuje codzienne obrazy PlanetScope dla danego okresu
    
    Args:
        start_date: data początkowa (str lub datetime)
        end_date: data końcowa (str lub datetime)
        aoi: geometria obszaru zainteresowania (GeoJSON)
        max_cloud_cover: maksymalne zachmurzenie (0.0-1.0)
    """
    
    # Konwersja dat
    if isinstance(start_date, str):
        start_date = datetime.fromisoformat(start_date)
    if isinstance(end_date, str):
        end_date = datetime.fromisoformat(end_date)
    
    # Utworzenie filtrów
    search_filter = data_filter.and_filter([
        data_filter.geometry_filter(aoi),
        data_filter.date_range_filter(
            field_name="acquired",
            gte=start_date,
            lte=end_date
        ),
        data_filter.range_filter(
            field_name="cloud_cover",
            gte=0.0,
            lte=max_cloud_cover
        ),
        data_filter.string_in_filter(
            field_name="quality_category",
            values=["standard"]
        )
    ])
    
    # Wyszukiwanie - zwraca generator
    results = pl.data.search(
        item_types=["PSScene"],
        search_filter=search_filter
    )
    
    return results

# Przykład użycia
start = "2024-12-01T00:00:00Z"
end = "2024-12-31T23:59:59Z"
results = search_daily_imagery(start, end, remedhus_aoi, max_cloud_cover=0.15)

# POPRAWIONE: Bezpośrednie iterowanie po generatorze
items = []
print("Wyszukiwanie obrazów...")
for item in results:
    items.append(item)
    if len(items) % 100 == 0:
        print(f"Znaleziono {len(items)} obrazów...")

print(f"\nŁącznie znaleziono {len(items)} obrazów")

# Wyświetl przykładowe informacje o pierwszym obrazie
if items:
    first = items[0]
    print(f"\nPrzykładowy obraz:")
    print(f"  ID: {first['id']}")
    print(f"  Data: {first['properties']['acquired']}")
    print(f"  Zachmurzenie: {first['properties']['cloud_cover']*100:.1f}%")
    print(f"  Typ instrumentu: {first['properties']['instrument']}")


Wyszukiwanie obrazów...


[planet-auth-python] Error loading auth token. Continuing with old configuration and token data. Load error: [Errno 2] No such file or directory: 'C:\\Users\\delve\\.planet\\planet-user\\token.json'
[planet-auth-python] Error obtaining new or refreshed auth token. Continuing with old configuration and token data. Refresh error: [Errno 2] No such file or directory: 'C:\\Users\\delve\\.planet\\planet-user\\token.json'


InvalidAPIKey: {"message": "Please provide valid credentials. See https://docs.planet.com/develop/authentication for help.", "errors": []}

## Tworzenie zamówienia

In [ ]:
def create_order_for_items(item_ids, order_name="REMEDHUS_daily"):
    """
    Tworzy zamówienie dla wybranych obrazów
    """
    
    # Wybór typu produktu
    # ortho_analytic_8b_sr - 8-kanałowe dane z korekcją atmosferyczną
    # ortho_analytic_4b_sr - 4-kanałowe dane z korekcją atmosferyczną
    
    request = order_request.build_request(
        name=order_name,
        products=[
            order_request.product(
                item_ids=item_ids,
                product_bundle='analytic_8b_sr_udm2',  # 8-band surface reflectance
                item_type='PSScene'
            )
        ],
        tools=[
            order_request.clip_tool(aoi=remedhus_aoi),  # przycięcie do AOI
            order_request.reproject_tool(
                projection="EPSG:4326",  # WGS84
                kernel="cubic"
            )
        ]
    )
    
    # Utworzenie zamówienia
    order = pl.orders.create_order(request)
    print(f"Zamówienie utworzone: {order['id']}")
    
    return order

# Przykład: wybierz pierwsze 10 obrazów
item_ids = [item['id'] for item in items[:10]]
order = create_order_for_items(item_ids)

## Pobieranie zamówienia

In [ ]:
def download_order(order_id, dest_dir='./remedhus_data'):
    """
    Czeka na przetworzenie i pobiera zamówienie
    """
    import os
    
    # Oczekiwanie na przetworzenie (może trwać kilka minut)
    print("Oczekiwanie na przetworzenie zamówienia...")
    pl.orders.wait(order_id, max_attempts=300, delay=30)
    
    # Pobieranie
    print(f"Pobieranie do {dest_dir}...")
    os.makedirs(dest_dir, exist_ok=True)
    pl.orders.download_order(order_id, directory=dest_dir, overwrite=True)
    
    print("Pobieranie zakończone!")

# Pobierz zamówienie
download_order(order['id'], dest_dir='./data/remedhus_2024')

In [ ]:
def download_daily_timeseries(start_date, end_date, aoi, output_dir):
    """
    Automatycznie pobiera całą serię czasową
    """
    import os
    from pathlib import Path
    
    # Wyszukaj obrazy
    print(f"Wyszukiwanie obrazów od {start_date} do {end_date}...")
    results = search_daily_imagery(start_date, end_date, aoi)
    items = list(results.items_iter(limit=5000))
    
    print(f"Znaleziono {len(items)} obrazów")
    
    # Grupuj po datach (może być kilka obrazów dziennie)
    from collections import defaultdict
    images_by_date = defaultdict(list)
    
    for item in items:
        date = item['properties']['acquired'][:10]  # YYYY-MM-DD
        images_by_date[date].append(item['id'])
    
    print(f"Pokrycie czasowe: {len(images_by_date)} dni")
    
    # Orders API ma limit 500 item_ids na zamówienie
    # Dziel na mniejsze partie
    batch_size = 500
    all_ids = [item['id'] for item in items]
    
    for i in range(0, len(all_ids), batch_size):
        batch = all_ids[i:i+batch_size]
        print(f"\nPrzetwarzanie partii {i//batch_size + 1}...")
        
        order = create_order_for_items(
            batch,
            order_name=f"REMEDHUS_batch_{i//batch_size + 1}"
        )
        
        download_order(
            order['id'],
            dest_dir=os.path.join(output_dir, f'batch_{i//batch_size + 1}')
        )

# Użycie
download_daily_timeseries(
    start_date="2024-01-01",
    end_date="2024-12-31",
    aoi=remedhus_aoi,
    output_dir="./data/remedhus_timeseries"
)
